# Train Networks

- Train SoftMax or Multi-BCE classifier for the EEG diagnosis classification
    - CAUEEG-task1 benchmark: Classification of **Normal**, **MCI**, and **Dementia** symptoms
    - CAUEEG-task2 benchmark: Classification of **Normal** and **Abnormal** symptoms
- `Weights and Biases` sweep is used for hyperparameter search

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Load some packages
import os
import json
from copy import deepcopy
import gc
import time

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import wandb

# custom package
from datasets.caueeg_dataset import *
from datasets.caueeg_script import *
from models import *
from train import *

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## Set the default configuration for building datatset

In [4]:
cfg_data = {}
cfg_data['device'] = device
cfg_data['task'] = 'task1'
cfg_data['dataset_path'] = r'local/dataset/02_Curated_Data_220419/'
cfg_data['file_format'] = 'memmap'  # 'feather', 'memmap'
cfg_data['load_event'] = False
cfg_data['latency'] = 200 * 10      # 10 seconds
cfg_data['crop_length'] = 200 * 10  # 10 seconds
cfg_data['longer_crop_length'] = 200 * 10 * 6 # 60 seconds
cfg_data['crop_multiple'] = 4
cfg_data['minibatch'] = 256
cfg_data['EKG'] = 'O'
cfg_data['photic'] = 'X'
cfg_data['input_norm'] = 'dataset'  # 'datatset', 'datapoint', 'no'
cfg_data['awgn'] = 5e-2
cfg_data['mgn'] = 1e-4
cfg_data['awgn_age'] = 5e-2

In [5]:
cfg_data_temp = deepcopy(cfg_data)
_ = build_dataset_for_train(cfg_data_temp, verbose=True)

in_channels = cfg_data_temp['preprocess_train'](next(iter(_[0])))['signal'].shape[1]
out_dims = len(cfg_data_temp['class_label_to_name'])

del _

transform: Compose(
    EegRandomCrop(crop_length=2000, length_limit=10000000, multiple=4, latency=2000, return_timing=False)
    EegDropChannels(drop_index=[20])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------

transform_longer: Compose(
    EegRandomCrop(crop_length=12000, length_limit=10000000, multiple=4, latency=2000, return_timing=False)
    EegDropChannels(drop_index=[20])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------


task config:
{'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'task_description': 'Classification of [Normal], [MCI], and [Dementia] '
                     'symptoms.',
 'task_name': 'CAUEEG-task1 benchmark'}

 ---------------------------------------------------------------------------------------------------- 

train_dataset[0].keys():
dict_keys

In [6]:
pprint.pprint(cfg_data_temp, width=250)

{'EKG': 'O',
 'age_mean': tensor([71.1920]),
 'age_std': tensor([9.5596]),
 'awgn': 0.05,
 'awgn_age': 0.05,
 'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'crop_length': 2000,
 'crop_multiple': 4,
 'dataset_name': 'CAUEEG dataset',
 'dataset_path': 'local/dataset/02_Curated_Data_220419/',
 'device': device(type='cuda'),
 'file_format': 'memmap',
 'input_norm': 'dataset',
 'latency': 2000,
 'load_event': False,
 'longer_crop_length': 12000,
 'mgn': 0.0001,
 'minibatch': 256,
 'photic': 'X',
 'preprocess_test': Sequential(
  (0): EegToDevice(device=cuda)
  (1): EegNormalizeAge(mean=tensor([71.1920]),std=tensor([9.5596]),eps=1e-08)
  (2): EegNormalizeMeanStd(mean=tensor([ 0.0570, -0.0301, -0.0042,  0.0441,  0.0291,  0.0292,  0.0372, -0.0056,
          -0.0383, -0.0292, -0.0727, -0.0123, -0.0068,  0.0350,  0.0457, -0.0048,
          -0.0180,  0.0505,  0.0254,  0.0020]),std=tensor([46.0711, 20.6705, 11.7231, 11.5120, 

-----

## Define Network Models

In [7]:
cfg_common_model = {'in_channels': in_channels, 
                    'out_dims': out_dims, 
                    'sequence_length': cfg_data['crop_length']}

cfg_model_pool = []

### 1D Tiny CNN

In [8]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Tiny-CNN'
cfg_model['generator'] = TinyCNN1D
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'fc_stages': 1,
 'final_pool': 'max',
 'generator': <class 'models.simple_cnn_1d.TinyCNN1D'>,
 'in_channels': 20,
 'model': '1D-Tiny-CNN',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

TinyCNN1D(
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv1d(20, 64, kernel_size=(35,), stride=(20,), padding=(17,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (final_pool): AdaptiveMaxPool1d(output_size=1)
  (fc_stage): Sequential(
    (0): Sequentia

### M5 model

In [9]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-M5'
cfg_model['generator'] = M5
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 256,
 'fc_stages': 1,
 'final_pool': 'max',
 'generator': <class 'models.simple_cnn_1d.M5'>,
 'in_channels': 20,
 'model': '1D-M5',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

M5(
  (conv1): Conv1d(20, 256, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(256, 256, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(256, 512, kernel_size=(9,), stride=(2,), padding=(4,), bias=False)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(512, 512, kernel_size=(9,), stride=(2,), padding=(4,), bias=False)
  (bn4): BatchNorm1

### 1D ResNet variants

In [10]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-18'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BasicBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BasicBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-18',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BasicBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
      (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)


In [11]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-50'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-50',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
    

In [12]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-101'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-101',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
  

### 1D ResNeXt variants

In [13]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-50'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 4
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-50',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc',
 'width_per_group': 4}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(128, 128, kernel_size=(9,), 

In [14]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-101'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-101',
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc',
 'width_per_group': 8}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(256, 256, kernel_size=(9,)

### 2D ResNet variants

In [15]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-18' # resnet-18 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = BasicBlock2D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.BasicBlock2D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'in_channels': 20,
 'model': '2D-ResNet-18',
 'out_dims': 3,
 'sequence_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BasicBlock2D(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv

In [16]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-50' # resnet-50 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
    'hop_length': 100,
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'in_channels': 20,
 'model': '2D-ResNet-50',
 'out_dims': 3,
 'sequence_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'hop_length': 100, 'n_fft': 200},
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): C

### 2D ResNeXt variants

In [17]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-50' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 4
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '2D-ResNeXt-50',
 'out_dims': 3,
 'sequence_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc',
 'width_per_group': 4}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): R

In [18]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-101' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '2D-ResNeXt-101',
 'out_dims': 3,
 'sequence_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc',
 'width_per_group': 8}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1):

#### CNN-Transformer

In [19]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-CNN-Transformer'
cfg_model['generator'] = CNNTransformer
cfg_model['fc_stages'] = 2
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256  #
cfg_model['n_encoders'] = 4  #
cfg_model['n_heads'] = 4     #
cfg_model['dropout'] = 0.2   #
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 256,
 'dropout': 0.2,
 'fc_stages': 2,
 'final_pool': 'max',
 'generator': <class 'models.cnn_transformer.CNNTransformer'>,
 'in_channels': 20,
 'model': '1D-CNN-Transformer',
 'n_encoders': 4,
 'n_heads': 4,
 'out_dims': 3,
 'sequence_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

CNNTransformer(
  (conv0): Conv1d(256, 512, kernel_size=(21,), stride=(6,), padding=(10,), bias=False)
  (norm0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv1): Conv1d(512, 1024, kernel_size=(9,), stride=(2,), padding=(4,), bias=False)
  (norm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (conv2): Conv1d(1024, 1024, kernel_size=(9,), stride=(2,), padding=(4,), bias=False)
  (norm2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=Tr

#### Summarize the loaded models

In [20]:
for cfg_model in cfg_model_pool:
    pprint.pp(cfg_model, width=150)
    print('\n' + '-' * 100 + '\n')

{'in_channels': 20,
 'out_dims': 3,
 'sequence_length': 2000,
 'model': '1D-M5',
 'generator': <class 'models.simple_cnn_1d.M5'>,
 'fc_stages': 1,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 256,
 'LR': None,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'sequence_length': 2000,
 'model': '1D-ResNet-18',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BasicBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'LR': None,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'sequence_length': 2000,
 'model': '1D-ResNet-50',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3

-----

## Default Configurations for Training

In [21]:
# training configurations
cfg_train = {}
cfg_train['iterations'] = (250000 * 32) // cfg_data['minibatch']
cfg_train['num_history'] = 500
cfg_train['lr_scheduler_type'] = 'transformer_style'
cfg_train['warmup_steps'] = 10000
cfg_train['weight_decay'] = 1e-2
cfg_train['mixup'] = 0.0 # 0 for no usage
cfg_train['criterion'] = 'cross-entropy' # 'cross-entropy', 'multi-bce'

cfg_train['device'] = device
cfg_train['save_model'] = True
cfg_train['draw_result'] = True
cfg_train['watch_model'] = False

-----

## Train

In [22]:
sweep_data = {}
sweep_data['crop_length'] = {
    'values': [
        200 * 14, #  5 sec
        200 * 10, # 10 sec
        200 * 15, # 15 sec
        200 * 20, # 20 sec
        200 * 30, # 30 sec
              ],
}

sweep_data['EKG'] = {
    'values': ['O', 'X'],
}

sweep_data['photic'] = {
    'values': ['O', 'X'],
}

sweep_data['awgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['mgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.1,
}

sweep_data['awgn_age'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

In [23]:
sweep_model = {}
sweep_model['model_index'] = { 
    'values': [i for i in range(len(cfg_model_pool))] 
}

sweep_model['fc_stages'] = { 
    'distribution': 'int_uniform',
    'min': 1,
    'max': 4,
}

sweep_model['use_age'] = { 
    'values': ['fc', 'conv', 'no']
}

sweep_model['first_dilation'] = { 
    'distribution': 'int_uniform',
    'min': 1,
    'max': 3,
}

sweep_model['base_channels'] = { 
    'values': [32, 64, 128]
}

sweep_model['dropout'] = {
    'values': [0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

# sweep_model['LR'] = {
#     'distribution': 'log_uniform',
#     'min': math.log(5e-5),
#     'max': math.log(2e-3)
# }

sweep_model['activation'] = {
    'values': ['relu', 'gelu', 'mish']
}

In [24]:
sweep_train = {}
# sweep_train['iterations'] = {
#     'values' : [150000]
# }

sweep_train['lr_scheduler_type'] = {
    'values': [
        # 'constant_with_decay',
        'transformer_style',
        'cosine_decay_with_warmup_half',
        'cosine_decay_with_warmup_one_and_half',
        'cosine_decay_with_warmup_two_and_half',
        'get_linear_with_warmup',
    ]

}

sweep_train['search_multiplier'] = {
    'values': [1, 2, 5]
}

sweep_train['weight_decay'] = {
    'distribution' : 'log_uniform_values',
    'min': 1e-5,
    'max': 1e-1
}

sweep_train['mixup'] = {
    'values': [0, 0.1, 0.2, 0.3]
}

sweep_train['criterion'] = {
    'values': ['cross-entropy', 'multi-bce']
}

In [25]:
sweep_config = {
    "entity": "ipis-mjkim",
    "name" : "my-sweep",
    "method": "random",
    "parameters": 
    {
        **sweep_data,
        **sweep_model,
        **sweep_train,
    }
}

sweep_id = wandb.sweep(sweep_config, project=f"caueeg-{cfg_data['task']}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 4z0y5nfl
Sweep URL: https://wandb.ai/ipis-mjkim/caueeg-task1/sweeps/4z0y5nfl


In [26]:
def train():
    # initialize the wandb log
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id

    with wandb_run:
        # collect some garbages
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
        # load default config
        config = {}
        cfg_model = cfg_model_pool[wandb.config.model_index]
        for k, v in {**cfg_data, **cfg_train, **cfg_model}.items():
            if k not in wandb.config:
                config[k] = v

        # to prevent callables from type-conversion to str
        config['history_interval'] = max(config['iterations'] // config['num_history'], 1)
        wandb.config.update(config)
        for k, v in wandb.config.items():
            if k not in config:
                config[k] = v

        # build dataset
        train_loader, val_loader, test_loader, test_loader_longer = build_dataset_for_train(config)
        
        # model in/out configuration
        config['in_channels'] = config['preprocess_train'](next(iter(train_loader)))['signal'].shape[1]
        config['out_dims'] = len(config['class_label_to_name'])
        
        # train the model
        model = train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, 
                                 config['preprocess_train'], config['preprocess_test'])
        
        # release memory
        del model
        del train_loader, val_loader
        del test_loader, test_loader_longer
        del config
        
    time.sleep(60)

In [ ]:
wandb.agent(sweep_id, function=train, count=100)

wandb: Agent Starting Run: 1zdx5e8u with config:
wandb: 	EKG: O
wandb: 	activation: relu
wandb: 	awgn: 0.18593777364754757
wandb: 	awgn_age: 0.2352738150386636
wandb: 	base_channels: 64
wandb: 	criterion: cross-entropy
wandb: 	crop_length: 3000
wandb: 	dropout: 0.4
wandb: 	fc_stages: 3
wandb: 	first_dilation: 3
wandb: 	lr_scheduler_type: cosine_decay_with_warmup_one_and_half
wandb: 	mgn: 0.03913565023485332
wandb: 	mixup: 0.3
wandb: 	model_index: 7
wandb: 	photic: X
wandb: 	search_multiplier: 5
wandb: 	use_age: fc
wandb: 	weight_decay: 0.0017755533024790095
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ipis-mjkim (use `wandb login --relogin` to force relogin)


************************************************************************************************************************
******************************                 2D-ResNet-50 train starts                  ******************************
************************************************************************************************************************



Run 1zdx5e8u errored: AttributeError("'ResNet2D' object has no attribute 'poo1'")
wandb: ERROR Run 1zdx5e8u errored: AttributeError("'ResNet2D' object has no attribute 'poo1'")
wandb: Agent Starting Run: 363ezvmp with config:
wandb: 	EKG: O
wandb: 	activation: gelu
wandb: 	awgn: 0.10627925467130488
wandb: 	awgn_age: 0.2220071812043113
wandb: 	base_channels: 32
wandb: 	criterion: cross-entropy
wandb: 	crop_length: 2800
wandb: 	dropout: 0.3
wandb: 	fc_stages: 2
wandb: 	first_dilation: 1
wandb: 	lr_scheduler_type: cosine_decay_with_warmup_two_and_half
wandb: 	mgn: 0.07018163906347655
wandb: 	mixup: 0.2
wandb: 	model_index: 3
wandb: 	photic: O
wandb: 	search_multiplier: 1
wandb: 	use_age: no
wandb: 	weight_decay: 0.00044222676795807055
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


************************************************************************************************************************
******************************                 1D-ResNet-101 train starts                 ******************************
************************************************************************************************************************
